In [8]:
# importing time and threading
import time
import threading
import math
from pynput.mouse import Button, Controller
import numpy as np
import cv2
import pyautogui
from PIL import Image 

# pynput.keyboard is used to watch events of
# keyboard for start and stop of auto-clicker

import pynput.keyboard
from pynput.keyboard import Listener, KeyCode, Key

In [9]:
fire = Button.left
right = Button.right
aim_fire = KeyCode(char='c')
stop_key = Key.esc

In [ ]:
np.set_printoptions(threshold=np.inf)

image = cv2.imread(r'C:\Users\yanghan9014\Desktop\auto clicker\test14.png')
# cv2.imshow('original', image)

original = image.copy()

image = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2HSV)

# image[400:600, 850:1400] = cv2.blur(image[400:600, 850:1400] ,(50,50))
# print(image)
# original_hsv_blurred = image.copy()

# loop over the boundaries

lower = np.array([0,0,0], dtype="uint8")
upper = np.array([0,0,0], dtype="uint8")

mask = cv2.inRange(image, lower, upper)
i=0
for (lower, upper) in boundaries:
#     create NumPy arrays from the boundaries
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    submask = cv2.inRange(image, lower, upper)
    cv2.imshow('submask{}'.format(i), submask)
    i+=1
    mask = mask + submask


kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
opening = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=3)

cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

area = 0
for c in cnts:
    area += cv2.contourArea(c)
    cv2.drawContours(original,[c], 0, (0,0,0), 2)

print(area)

# cv2.imshow('mask', mask)
# cv2.imshow('original', original)
# cv2.imshow('original_hsv_blurred', original_hsv_blurred)

cv2.imshow('opening', opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
boundaries = [
    ([0, 0, 70], [30, 12, 255]),
    ([3, 140, 25], [15, 230, 100]),
    ([10, 140, 60], [20, 165, 250]),
]

In [ ]:
circles = cv2.HoughCircles(opening, cv2.HOUGH_GRADIENT, 1, 50,
                             param1=50, param2=10, minRadius=12,maxRadius=70)

try:
    circles = np.uint16(np.around(circles))
    circles = circles[0,:][:,0:2]
#     print(circles)
    
    dist = np.linalg.norm(circles - (729, 380), axis = 1)
    target = circles[np.argmin(dist)]
    print(target)
    
    cv2.circle(original, target, 5, (255,0,0), 10)
    for i in circles:
        cv2.circle(original, i, 20, (0,255,0), 2)
        cv2.circle(original, i, 2, (0,0,255), 3)
        
except:
    pass

cv2.imshow('circle detector',original)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
import mss
import mss.tools

save_index = 1
def coords_from_mid():
    t0 = time.time()

    with mss.mss() as sct:
        # The screen part to capture
        region = {'top': 230, 'left': 200, 'width': 1400, 'height': 600}

        # Grab the data
        image = sct.grab(region)

#         # Save to the picture file
#         mss.tools.to_png(image.rgb, image.size, output='dummy.png')
        
#     image = pyautogui.screenshot(region=(230,200, 1400, 600))
    
    dt1 = time.time() - t0
    t1 = time.time()
    
    original = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
#     original[400:600, 850:1400] = cv2.blur(original[400:600, 850:1400] ,(50,50))
    
    image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2HSV)
    image[400:600, 850:1400] = cv2.blur(image[400:600, 850:1400] ,(50,50))
    
    lower = np.array([0,0,0], dtype="uint8")
    upper = np.array([0,0,0], dtype="uint8")

    mask = cv2.inRange(image, lower, upper)
    i=0
    for (lower, upper) in boundaries:
    #     create NumPy arrays from the boundaries
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        submask = cv2.inRange(image, lower, upper)
#         cv2.imshow('submask{}'.format(i), submask)
        i+=1
        mask = mask + submask
    
    dt2 = time.time() - t1
    t2 = time.time()
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=3)
    
    circles = cv2.HoughCircles(opening, cv2.HOUGH_GRADIENT, dp=3, minDist=250, param1=110, param2=60, minRadius=7,maxRadius=150)
    dt3 = time.time() - t2
    t3 = time.time()
    
    
    if(type(circles) == np.ndarray):
#         circles = np.uint16(np.around(circles))
        circles = circles[0,:][:,0:2]
    #         print(circles)

    #         dist = np.linalg.norm(circles - (729, 380), axis = 1
        dist = np.linalg.norm(circles - (729, 380), axis = 1)
        target = circles[np.argmin(dist)] - (729, 380)




#         cv2.circle(original, target + (729, 380), 5, (255,0,0), 10)
#         for i in circles:
#             cv2.circle(original, i, 20, (0,255,0), 2)
#             cv2.circle(original, i, 2, (0,0,255), 3)

#         global save_index
#         cv2.imwrite(r'C:\Users\yanghan9014\Desktop\auto clicker\playtest_{}.png'.format(save_index), original)
#         save_index += 1

        dt4 = time.time() - t3
        dt = time.time() - t0
        print("targets found: {}, selected {}, time {:.3f}, dt1 {:.3f}, dt2 {:.3f}, dt3 {:.3f}, dt4 {:.3f}".format(dist.size, target, dt, dt1, dt2,dt3, dt4))
        return target
    else:
#         cv2.imwrite(r'C:\Users\yanghan9014\Desktop\auto clicker\playtest_{}.png'.format(save_index), original)
#         save_index += 1
        return None
        
    return None
    

In [14]:
mouse = Controller()
keyboard = pynput.keyboard.Controller()
mouse_speed = 0.7

# on_press method takes
# key as argument
def on_press(key):
    global listener
    
    if key == aim_fire:
#         listener.stop()
#         print(mouse.position)
        target = coords_from_mid()
        if(type(target) == np.ndarray):
            mouse.position = mouse.position + mouse_speed * target
            mouse.click(fire)
            print("hit")
#             keyboard.release(aim_fire)
            keyboard.press('r')
        else:
#             keyboard.release(aim_fire)
            print("no target")
            
        target = None
#         with pynput.keyboard.Listener(on_press=on_press) as listener:
#             listener.join()
            
    elif key == stop_key:
        listener.stop()


with pynput.keyboard.Listener(on_press=on_press) as listener:
    listener.join()
# listener = pynput.keyboard.Listener(on_press=on_press)
# listener.start()

targets found: 4, selected [-193.5    8.5], time 0.092, dt1 0.041, dt2 0.018, dt3 0.034, dt4 0.000
hit
targets found: 1, selected [283.5 164.5], time 0.143, dt1 0.070, dt2 0.030, dt3 0.044, dt4 0.000
hit
targets found: 1, selected [133.5 -45.5], time 0.108, dt1 0.043, dt2 0.030, dt3 0.031, dt4 0.004
hit
targets found: 4, selected [19.5 89.5], time 0.133, dt1 0.060, dt2 0.030, dt3 0.043, dt4 0.000
hit
targets found: 2, selected [-43.5 -21.5], time 0.143, dt1 0.080, dt2 0.031, dt3 0.032, dt4 0.000
hit
targets found: 3, selected [-124.5 -339.5], time 0.160, dt1 0.064, dt2 0.035, dt3 0.060, dt4 0.000
hit
targets found: 5, selected [328.5  71.5], time 0.085, dt1 0.040, dt2 0.020, dt3 0.020, dt4 0.005
hit
